In [1]:
import pandas as pd
import numpy as np
import polars as pl
import json

import yfinance as yf
import time

from data.cons_data import get_cons
from pairs_identification import cointegration_pairs
from clustering import Clustering

In [2]:
etf = "QQQ"

cons = get_cons(etf=etf)


In [3]:
cons_date = cons.read()
cons_date["2024-12-31"]

['AAPL',
 'ABNB',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'AEP',
 'AMAT',
 'AMD',
 'AMGN',
 'AMZN',
 'ANSS',
 'APP',
 'ARM',
 'ASML',
 'AVGO',
 'AXON',
 'AZN',
 'BIIB',
 'BKNG',
 'BKR',
 'CCEP',
 'CDNS',
 'CDW',
 'CEG',
 'CHTR',
 'CMCSA',
 'COST',
 'CPRT',
 'CRWD',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTSH',
 'DASH',
 'DDOG',
 'DXCM',
 'EA',
 'EXC',
 'FANG',
 'FAST',
 'FTNT',
 'GEHC',
 'GFS',
 'GILD',
 'GOOG',
 'GOOGL',
 'HON',
 'IDXX',
 'INTC',
 'INTU',
 'ISRG',
 'KDP',
 'KHC',
 'KLAC',
 'LIN',
 'LRCX',
 'LULU',
 'MAR',
 'MCHP',
 'MDB',
 'MDLZ',
 'MELI',
 'META',
 'MNST',
 'MRVL',
 'MSFT',
 'MSTR',
 'MU',
 'NFLX',
 'NVDA',
 'NXPI',
 'ODFL',
 'ON',
 'ORLY',
 'PANW',
 'PAYX',
 'PCAR',
 'PDD',
 'PEP',
 'PLTR',
 'PYPL',
 'QCOM',
 'REGN',
 'ROP',
 'ROST',
 'SBUX',
 'SNPS',
 'TEAM',
 'TMUS',
 'TSLA',
 'TTD',
 'TTWO',
 'TXN',
 'VRSK',
 'VRTX',
 'WBD',
 'WDAY',
 'XEL',
 'ZS']

In [4]:
dates = list(cons_date.keys())

In [5]:
start = "2021-09-30"
end = "2021-12-31"

In [6]:
year_to_read = list(set([start[:4], end[:4]]))

if len(year_to_read) > 1:
    df = pl.concat(
        [
            pl.read_parquet(
                f"C:/Users/edmun/OneDrive/Desktop/Quantitative Trading Strategies/Project/qts/data/{year_to_read[0]}.parquet",
                columns=[
                    "ts_event",
                    "close",
                    "symbol",
                ],
            ),
            pl.read_parquet(
                f"C:/Users/edmun/OneDrive/Desktop/Quantitative Trading Strategies/Project/qts/data/{year_to_read[1]}.parquet",
                columns=[
                    "ts_event",
                    "close",
                    "symbol",
                ],
            ),
        ],
        how="vertical",
    ).lazy()
else:
    df = pl.read_parquet(
        f"C:/Users/edmun/OneDrive/Desktop/Quantitative Trading Strategies/Project/qts/data/{year_to_read[0]}.parquet",
        columns=[
            "ts_event",
            "close",
            "symbol",
        ],
    ).lazy()

df = (
    df.with_columns(date=pl.col("ts_event").str.strptime(pl.Datetime))
    .with_columns(
        date=pl.col("date").dt.date(),
        time=pl.col("date").dt.time(),
    )
    .filter(
        (
            pl.col("date").is_between(
                lower_bound=pl.lit(start).str.strptime(pl.Date, "%Y-%m-%d")
                - pl.duration(days=10),  # filter few days behind to get the ffill price
                upper_bound=pl.lit(end).str.strptime(pl.Date, "%Y-%m-%d"),
            )
        )
        & (pl.col("symbol").is_in(cons_date[start]))
    )
    .collect()
    .pivot(on="symbol", index=["date", "time"], values="close")
    .sort(by=["date", "time"])
    .fill_null(strategy="forward")
    .filter(pl.col("date") >= pl.lit(start).str.strptime(pl.Date, "%Y-%m-%d"))
    # .with_columns(pl.col("time").dt.to_string().str.to_datetime("%H:%M:%S"))
    # .upsample(time_column="time", every="5m", group_by="date", maintain_order=True)
)

In [7]:
df.group_by('date').count().to_pandas()['count'].value_counts() # to checc

C:\Users\edmun\AppData\Local\Temp\ipykernel_19968\2237246335.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  df.group_by('date').count().to_pandas()['count'].value_counts() # to checc


count
900    33
960    15
958     4
959     4
957     2
899     2
951     1
948     1
898     1
780     1
897     1
Name: count, dtype: int64

In [8]:
df.null_count().sum_horizontal()

sum
u32
0


In [9]:
df = df.fill_null(strategy="forward").drop_nulls()

In [10]:
df

date,time,AAPL,INTC,TSLA,MSFT,ZM,MU,NVDA,NTES,BIDU,MRNA,PDD,CSCO,FB,PEP,AMZN,GOOGL,AMD,ASML,QCOM,PYPL,GOOG,SBUX,VRSN,JD,ADBE,EA,NFLX,WBA,AMAT,BIIB,INTU,DOCU,HON,ISRG,PTON,…,NXPI,CMCSA,SNPS,LULU,MTCH,TXN,SPLK,KDP,SWKS,CPRT,WDAY,CERN,PCAR,ROST,OKTA,CSX,DLTR,SIRI,TEAM,PAYX,CHKP,KLAC,DXCM,MCHP,FOXA,FAST,CTAS,SGEN,ADP,ORLY,IDXX,CTSH,ANSS,INCY,XEL,VRSK,CDW
date,time,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2021-09-30,08:00:00,143.9,53.73,784.35,286.32,258.0,71.88,207.65,84.12,152.11,383.66,89.15,55.83,340.6,152.35,3325.0,2693.0,100.9,776.72,129.66,260.45,2697.81,112.5,206.21,72.01,580.0,138.53,602.7,48.96,129.94,286.29,546.59,257.5,214.92,992.9,85.8,…,199.95,55.94,302.62,410.35,154.64,194.26,138.15,34.96,166.34,141.82,249.51,71.13,81.17,115.12,233.33,30.25,101.11,6.1,387.6,108.47,114.83,336.0,545.3,153.1,40.25,52.82,389.26,164.25,200.16,628.0,629.55,75.58,341.72,68.67,62.67,202.37,185.39
2021-09-30,08:01:00,144.08,53.85,784.35,285.99,258.0,71.88,207.84,83.91,152.11,384.13,90.0,55.83,342.61,152.35,3326.43,2702.5,101.45,776.57,129.66,261.25,2697.81,112.14,206.8,72.01,580.0,138.53,602.28,48.96,130.13,286.29,546.59,257.5,214.92,992.9,85.8,…,199.95,55.94,302.62,410.35,154.64,194.26,138.15,34.96,166.34,141.82,249.51,71.13,81.17,115.12,233.33,30.25,101.11,6.1,387.6,108.47,114.83,336.0,545.3,153.1,40.25,52.82,389.26,164.25,200.16,628.0,629.55,75.58,341.72,68.67,62.67,202.37,185.39
2021-09-30,08:02:00,144.0,53.81,783.72,286.2,260.0,71.88,207.82,83.91,152.11,384.13,90.59,55.83,342.75,152.35,3326.43,2713.98,101.4,776.5,129.66,261.25,2713.66,112.14,206.8,72.75,580.0,138.53,603.0,48.96,130.13,286.29,546.59,257.5,214.92,992.9,85.8,…,199.95,55.94,302.62,410.35,154.64,194.26,138.15,34.96,166.34,141.82,249.51,71.13,81.17,115.12,233.33,30.25,101.0,6.1,387.6,108.47,114.83,336.0,545.3,153.1,40.25,52.82,389.26,164.25,200.16,628.0,629.55,75.58,341.72,68.67,62.67,202.37,185.39
2021-09-30,08:03:00,144.01,53.81,784.3,286.2,260.0,71.88,207.75,83.91,152.11,384.13,90.08,55.83,342.75,152.35,3327.4,2713.9,101.23,775.68,129.66,261.25,2714.0,112.14,206.8,72.68,580.0,138.53,604.0,48.96,130.13,286.29,546.59,257.5,214.92,992.9,85.8,…,199.95,55.94,302.62,410.35,154.64,194.26,138.15,34.96,166.34,141.82,249.51,71.13,81.17,115.12,233.33,30.25,101.0,6.1,387.6,108.47,114.83,336.0,545.3,153.1,40.25,52.82,389.26,164.25,200.16,628.0,629.55,75.58,341.72,68.67,62.67,202.37,185.39
2021-09-30,08:04:00,144.0,53.81,784.3,286.2,259.4,71.88,207.75,84.0,152.11,384.13,90.08,55.83,342.75,152.35,3325.7,2713.9,101.23,776.16,129.66,261.25,2714.0,112.45,206.8,72.45,580.0,138.53,602.78,48.96,130.19,286.29,546.59,257.5,214.92,992.9,85.8,…,199.95,55.94,302.62,410.35,154.64,194.26,138.15,34.96,166.34,141.82,249.51,71.13,81.17,115.12,233.33,30.25,101.0,6.1,387.6,108.47,114.83,336.0,545.3,153.1,40.25,52.82,389.26,164.25,200.16,628.0,629.55,75.58,341.72,68.67,62.67,202.37,185.39
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2021-12-31,23:55:00,177.68,51.51,1060.2,336.35,184.0,93.3,294.16,103.22,148.8,254.09,58.31,63.4,337.4,173.81,3334.33,2900.9,143.94,796.14,183.2,188.41,2896.5,116.89,253.99,70.21,567.15,131.9,604.0,52.33,157.65,239.0,640.62,152.15,209.05,358.0,35.78,…,228.0,50.63,368.5,391.8,132.51,189.24,115.35,36.91,155.1,151.68,273.2,93.09,88.4,114.28,224.2,37.68,140.03,6.37,381.3,136.5,116.56,430.68,536.95,87.25,36.9,64.34,443.17,154.91,246.58,706.23,658.46,88.96,402.68,73.4,67.3,229.62,204.78
2021-12-31,23:56:00,177.68,51.51,1060.11,336.35,184.0,93.3,294.16,103.22,148.8,254.09,58.31,63.4,337.4,173.81,3334.0,2900.9,143.94,796.14,183.2,188.41,2896.5,116.89,253.99,70.21,567.15,131.9,604.0,52.33,157.65,239.0,640.62,152.15,209.05,358.0,35.78,…,228.0

In [11]:
c = Clustering(df=df)
scores = c.kmeans_clustering(min_clusters=5, max_clusters=20)

c:\Users\edmun\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\edmun\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

1


c:\Users\edmun\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\edmun\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [12]:
c.cluster_pairs

{2: ['AAPL',
  'MSFT',
  'CSCO',
  'PEP',
  'AMZN',
  'GOOGL',
  'GOOG',
  'SBUX',
  'VRSN',
  'WBA',
  'INTU',
  'VRTX',
  'COST',
  'EXC',
  'MNST',
  'MDLZ',
  'GILD',
  'AEP',
  'AMGN',
  'KDP',
  'CPRT',
  'PCAR',
  'CSX',
  'SIRI',
  'PAYX',
  'CHKP',
  'FAST',
  'CTAS',
  'ADP',
  'ORLY',
  'CTSH',
  'ANSS',
  'XEL',
  'VRSK',
  'CDW'],
 4: ['INTC',
  'BIDU',
  'FB',
  'ASML',
  'ADBE',
  'EA',
  'NFLX',
  'HON',
  'ATVI',
  'ADSK',
  'TMUS',
  'BKNG',
  'ADI',
  'MAR',
  'ILMN',
  'FOX',
  'EBAY',
  'KHC',
  'REGN',
  'FISV',
  'ALGN',
  'CHTR',
  'CMCSA',
  'LULU',
  'TXN',
  'SWKS',
  'WDAY',
  'ROST',
  'DXCM',
  'FOXA',
  'SGEN',
  'IDXX',
  'INCY'],
 1: ['TSLA',
  'MU',
  'NVDA',
  'NTES',
  'AMD',
  'QCOM',
  'AMAT',
  'LRCX',
  'CDNS',
  'XLNX',
  'AVGO',
  'NXPI',
  'SNPS',
  'CERN',
  'DLTR',
  'KLAC'],
 0: ['ZM',
  'PYPL',
  'JD',
  'BIIB',
  'CRWD',
  'MELI',
  'MTCH',
  'SPLK',
  'OKTA',
  'TEAM'],
 5: ['MRNA', 'PDD', 'DOCU', 'PTON'],
 3: ['ISRG', 'MCHP']}

In [13]:
find_pairs = cointegration_pairs(
    df=df.select(pl.all().exclude(["date", "time"])),
    p_val_cutoff=0.05,
    cluster_pairs=c.cluster_pairs,
)

In [14]:
find_pairs.identify_pairs()

In [15]:
find_pairs.pairs

{('ISRG', 'PTON'): -3.011526121152664,
 ('TSLA', 'MSFT'): -3.0500958801708338,
 ('BIIB', 'INTU'): -3.8292223453699488,
 ('DOCU', 'ISRG'): -3.0521023103493463,
 ('HON', 'ISRG'): -2.969683871915732,
 ('GOOG', 'SBUX'): -3.7451642847380437,
 ('NFLX', 'WBA'): -3.1511828538793845,
 ('AMZN', 'QCOM'): -3.0725009501441907,
 ('ZM', 'NVDA'): -2.9085279628011467,
 ('INTU', 'HON'): -3.3980019298508184,
 ('QCOM', 'VRSN'): -2.887255609818266,
 ('AAPL', 'MSFT'): -2.9581629615762224,
 ('NFLX', 'AMAT'): -3.468229102763313,
 ('QCOM', 'JD'): -3.143517997142986,
 ('ADBE', 'BIIB'): -3.2325129448360834,
 ('WBA', 'DOCU'): -3.209383210386669,
 ('BIDU', 'FB'): -3.897722265156062,
 ('QCOM', 'ADBE'): -3.759130162358626,
 ('ADBE', 'INTU'): -4.6398103380498465,
 ('BIIB', 'HON'): -4.049890640859305,
 ('INTU', 'PTON'): -3.192680146167446,
 ('AMZN', 'JD'): -3.674041116318386,
 ('AMAT', 'DOCU'): -3.880469148866556,
 ('AMZN', 'ADBE'): -3.279675633262143,
 ('TSLA', 'NTES'): -3.064604468464193,
 ('GOOG', 'JD'): -2.8868325

In [16]:
trade_this = find_pairs.get_top_pairs()

In [17]:
trade_this

[('MU', 'NVDA'),
 ('NFLX', 'ISRG'),
 ('ADBE', 'INTU'),
 ('FB', 'ISRG'),
 ('BIDU', 'ISRG'),
 ('BIDU', 'AMAT'),
 ('TSLA', 'QCOM'),
 ('GOOG', 'TMUS'),
 ('AMD', 'HON'),
 ('AAPL', 'HON'),
 ('TSLA', 'ADBE'),
 ('BIIB', 'HON'),
 ('AMZN', 'VRTX'),
 ('ASML', 'QCOM'),
 ('MRNA', 'ADBE'),
 ('GOOGL', 'INTU'),
 ('GOOGL', 'ADBE'),
 ('ADBE', 'PTON'),
 ('BIDU', 'FB'),
 ('AMAT', 'ISRG')]